<hr/>

# EN.520.637 Foundations of Reinforcement Learning

<hr/>

<h1><font color="darkblue">Lab 4: Multi-armed Bandit and Monte Carlo Method  </font></h1>



## Deadline
11:59 pm Nov 5th, 2021 

##  Content
1. Multi-armed Bandit
2. Monte Carlo Method


In [ ]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

## Problem Statement [P 21, Ch 2.3, Sutton]

#### Define a 10-armed bandit problem in which the action values $q_*(a)$, $a = 1,...,10$, are samples from a standard norm distribution, i.e. Gaussian distribution with mean $= 0$ and variance $ = 1$. Then, when selected $A_t$ at time step $t$, the actual reward, $R_t$ is selected from a Gaussian distribution with mean = $q_*(A_t)$ and variance = 1.

## Problem 1.  Greedy and $\epsilon$-greedy algorithm

1. Implement a function/functions that run this game 2000 times with $\epsilon$-greedy algorithm. Your function/functions should take $\epsilon$ as one of the inputs and output:
<br>   (a) average reward at each time step
<br>   (b) percentage of optimal action at each time step. (optimal action is defined by $a^* = arg\underset{a}max           \, q^*(a)$ )
2. Call your function/functions to generate the average reward and percentage of optimal action at each time step with:
<br>   (a) Greedy-algorithm 
<br>   (b) $\epsilon$-greedy algorithm, $\epsilon=0.01$ 
<br>   (c) $\epsilon$-greedy algorithm, $\epsilon=0.1$. 
3. Plot the average reward and percentage of optimal action of those three cases and compare with [P 23 Fig 2.2 Sutton].

### 1.

### 2.

### 3.

## Problem 2.  UCB Action Selection

1. Implement a function/functions that run this game 2000 times with UCB Action Selection algorithm. Your function/functions should take $c$ as one of the inputs and output:
<br>   - average reward at each time step.
2. Call your function/functions to generate the average reward at each time step with:
<br>   - UCB Action Selection algorithm, $c = 2$.
3. Plot the average reward of 2.2 and 1.2c, then compare with [P 28 Fig 2.4 Sutton].

### 1.

### 2.

### 3.

## 3. Monte Carlo Method  (CartPole-v1 environment)

### 3.1 CartPole Introduction

We now apply Monte Carlo Method for CartPole problem. 


1. A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. 

0. The system is controlled by applying a force of +1 or -1 to the cart. 

0. The pendulum starts upright, and the goal is to prevent it from falling over. 

0. A reward of +1 is provided for every timestep that the pole remains upright. 

0. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.

0. For more info (See [SOURCE ON GITHUB](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)).

The following examples show the basic usage of this testing environment: 



### 3.1.1 Episode initialization and Initial Value

In [ ]:
import gym

In [ ]:
env = gym.make('CartPole-v0')
observation = env.reset() ##Initial an episode

print("Inital observation is {}".format(observation))

print("\nThis means the cart current position is {}".format(observation[0]), end = '')
print(" with velocity {},".format(observation[1]))

print("and the pole current angular position is {}".format(observation[2]), end = '')
print(" with angular velocity {},".format(observation[3]))


### 3.1.2 Take actions


Use env.step(action) to take an action

action is an integer from 0 to 1

0: "Left"; 1: "Right"

In [ ]:
print("Current observation is {}".format(observation))

action = 0 #go left
observation, reward, done, info = env.step(action) # simulate one step

print("\nNew observation is {}".format(observation))
print("Step reward is {}".format(reward))
print("Did episode just ends? -{}".format(done)) # episode ends when 3.1(6) happens


### 3.1.3 Simulate multiple episodes

(You may uncomment those lines to see an animation. However, it will not work for JupyterHub since the animation requires GL instead of webGL. If you have Jupyter notebook localy on your computer, this version of code will work through a virtual frame.)

In [ ]:
env = gym.make('CartPole-v0')
observation = env.reset()
total_reward = 0
ep_num = 0
# img = plt.imshow(env.render(mode='rgb_array')) 


for _ in range(1000):
    #     img.set_data(env.render(mode='rgb_array')) 
    #     display.display(plt.gcf())
    #     display.clear_output(wait=True)
    
    action = env.action_space.sample()     # this takes random actions
    observation, reward, done, info = env.step(action) 
       
    total_reward += reward
    


    if done:                               # episode just ends
        observation = env.reset()          # reset episode
        ep_num += 1

print("Average reward per episode is {}".format(total_reward/ep_num))
env.close()


### 3.1.4 States Discretization 

The class DiscreteObs() discretizes the observation space into discrete state space, based on numpy.digitize (Please read its description in https://numpy.org/doc/stable/reference/generated/numpy.digitize.html) 

Discretization of observation space is necessary for tabular methods. You can use DiscreteObs() or any other library for discretizing the observation space. 

In [ ]:
class DiscretObs():
    
    
    def __init__(self, bins_list):
        self._bins_list = bins_list
        
        self._bins_num = len(bins_list)
        self._state_num_list = [len(bins)+1 for bins in bins_list]
        self._state_num_total = np.prod(self._state_num_list)
    
    def get_state_num_total(self):
        
        return self._state_num_total
    
    def obs2state(self, obs):
        
        if not len(obs)==self._bins_num:
            raise ValueError("observation must have length {}".format(self._bins_num))
        else:
            return [np.digitize(obs[i], bins=self._bins_list[i]) for i in range(self._bins_num)]
        
    def obs2idx(self, obs):
        
        state = self.obs2state(obs)
        
        return self.state2idx(state)
    
    def state2idx(self, state):
        
        idx = 0
        for i in range(self._bins_num-1,-1,-1):
            idx = idx*self._state_num_list[i]+state[i]
        
        return idx
    
    def idx2state(self, idx):
        
        state = [None]*self._bins_num
        state_num_cumul = np.cumprod(self._state_num_list)
        for i in range(self._bins_num-1,0,-1):
            state[i] = idx/state_num_cumul[i-1]
            idx -=state[i]*state_num_cumul[i-1]
        state[0] = idx%state_num_cumul[0]
        
        return state

# Recommended Discretization for Carpole-v1 when using Monte-Carlo methods
bins_pos = np.linspace(-2.4,2.4,40)        # position
bins_d_pos = np.linspace(-3,3,5)           # velocity
bins_ang = np.linspace(-0.2618,0.2618,40)  # angle
bins_d_ang = np.linspace(-0.3,0.3,5)       # angular velocity

dobs = DiscretObs([bins_pos,bins_d_pos,bins_ang,bins_d_ang])
observation = env.reset()

state = dobs.obs2state(observation)
idx = dobs.obs2idx(observation)

print("Current position of the cart is {:.4f}\n".format(observation[0]))
print("Current velocity of the cart is {:.4f}\n".format(observation[1]))
print("Current angular position of the pole is {:.4f} rad\n".format(observation[2]))
print("Current angular velocity of the pole is {:.4f} rad\n".format(observation[3]))

print("which are mapped to state {}, with corresponding index {}".format(state,idx))


### 3.2 On-policy first-visit MC control

1. Implement "On-policy first-visit MC control" algorithum in [Ch 5.4 Sutton] to choose optimal actions
2. Simulate this algorithum for 40000 episodes.
3. Divide the previous 40000 episodes into 20 sets. Plot average rewards for each sets. (i.e. plot average rewards for the first 2000 episodes, the second 2000 episodes, ..., and the 15th 2000 episodes.) 
4. Use greedy policy of the trained Q function to control the carpole for 100 episode, plot the accumulate rewards over 100 episode

In [ ]:
## Suggested functions (Feel free to modify existing and add new functions)

def get_action(current_state, Q, epsilon):
    
    # Choose optimal action based on current state and Q
    #
    # input:  current state,  (array) 
    #         Q,              (array)  
    #         epsilonn,       (float)  
    # output: action
    #         
    return action



def update_Q(Q, observation_list, action_list):
    # Update Q at the end of each episode
    #
    # input:  current Q, (array) 
    #         observation_list,       (array)  states observed in this episode
    #         action_list,       (array)  actions took in this spisode
    # output: Updated Q
    #         

        
    return Q


## Suggested flow (Feel free to modify and add)

# parameters for epsilon-greedy algorithm, when epsilon_decay_rate=1, the algorothm implement a fixed 
# epsilon value as epsilon_start, you can choose either fixed epsilon or decaying epsilon

# epsilon_start = 0.3
# epsilon_decay_rate = 0.97

set_num = 30
s = 0
env = gym.make('CartPole-v1')
observation = env.reset()

#epsilon = epsilon_start   # set epsilon

while 1:
    
    
    current_state =                             # discretize the observation space
    
    action = get_action(current_state,Q,epsilon)# pick action by epsilon greedy policy
    
    observation, reward, done, info = env.step(action) # simulate one step
    
    if done:  # end of epsode
        Q = update_Q(Q, observation_list, action_list) # update Q for past observations in the episode
        
        ep_num += 1
        
        if  np.mod(ep_num,2000)==0: # end of every set of episode
            
            #epsilon = epsilon*epsilon_decay_rate     # update epsilon
            s+=1
            
            if s == set_num:
                break
env.close()

In [ ]:
# check your result here (Feel free to modify)
# the result_mc should be a (set_num, )-numpy array that records the average reward of a set of episodes 

# put your result here
font = {'weight' : 'bold',
        'size'   : 15}
matplotlib.rc('font', **font)

figure(figsize=(12,6))
ax = subplot(1,1,1)
ax.plot(range(0,set_num), result_mc, linewidth=2, color='g')

plt.ylabel("Set average reward");
plt.xlabel("Set number (2000 episodes simulated in each set)");

In [ ]:
# Use greedy policy of the trained Q function to control the carpole for 100 episode, 
# and plot the total reward received in each episode
## Suggested flow (Feel free to modify and add)

env = gym.make('CartPole-v1')
observation = env.reset()

count = 0
while 1:
    
    current_state =                             # discretize the observation space
    
    action =                                    # choose action by greedy policy of the trained Q
    
    observation, reward, done, info = env.step(action)
    
    
    if done:
        
        observation = env.reset()
        count +=1
        
        total_reward_mc =                                # record the total reward until this episode
        
        if count==100:
            break
        
font = {'weight' : 'bold',
        'size'   : 15}
matplotlib.rc('font', **font)

figure(figsize=(12,6))
ax = subplot(1,1,1)
ax.plot(range(100), total_reward_mc)

plt.ylabel("Accumulate Reward");
plt.xlabel("Episode");
    